# 載入套件

- Windows 用戶使用 tomorrow 套件時會報錯誤，修正方法如下
http://www.shangdixinxi.com/detail-1123784.html

In [4]:
# pip install grequests
# pip install aiohttp
# !pip install httpx

In [3]:
import requests
# from grequests import AsyncRequest
# import grequests
import asyncio
import aiohttp
import httpx
from bs4 import BeautifulSoup
import pandas as pd
import time
# from tomorrow import threads
headers = {'user-agent':'GoogleBot'}

# 設定查詢關鍵字與數量

In [5]:
keyword = '5G'
num = 10

# 爬取資料

## 收集新聞連結清單
當 requeest 回傳0筆資料是打破迴圈

In [6]:
Links = []
for page in range(num):
    url = 'https://udn.com/api/more?page={}&id=search:{}&channelId=2&type=searchword'.format(page+1, keyword)
    resp = requests.get(url, headers=headers)
    Lists = resp.json()['lists']
    print('There are {} news in page {}.'.format(len(Lists), page+1))
    if len(Lists) == 0:
        break
    for List in Lists:
        Links.append(List['titleLink'])
Links = list(set(Links))
print('There are {} Links.'.format(len(Links)))

There are 20 news in page 1.
There are 20 news in page 2.
There are 20 news in page 3.
There are 20 news in page 4.
There are 20 news in page 5.
There are 20 news in page 6.
There are 20 news in page 7.
There are 20 news in page 8.
There are 20 news in page 9.
There are 20 news in page 10.
There are 200 Links.


## 爬取新聞資料
resp 回來通常會包含很多咨詢，除了內文外還有網頁框架、推薦的新聞、廣告等資訊，我們可以先選擇要保留的資訊才不會佔用太多記憶體

### request

In [4]:
for link in Links:
    requests.get(link)

### grequest

In [7]:
rs = (grequests.get(link) for link in Links)

In [8]:
rs2 = grequests.map(rs)
# 1m38s

In [9]:
rs2

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 None,
 <Response [200]>,
 <Response [200]>,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Res

### asyncio

In [4]:
import httpx

async def get_async(url):
    async with httpx.AsyncClient() as client:
        return await client.get(url)

# Note that you need an async context to use `await`.
await asyncio.gather(*map(get_async, Links))

RuntimeError: This event loop is already running

In [11]:
loop.is_running()

True

In [10]:
loop.

<_WindowsSelectorEventLoop running=True closed=False debug=False>

In [5]:
loop = asyncio.get_event_loop()
async def send_req(url):
    resp = await loop.run_in_executor(None,requests.get,url)
    soup = BeautifulSoup(resp.text)
    print(soup.find('h1').text)

tasks = []

for link in Links:
    task = loop.create_task(send_req(link))
    tasks.append(task)

In [ ]:
loop.run_until_complete(asyncio.gather(tasks))

In [8]:
loop.is_running()

True

In [17]:
tmp = tasks[0]

In [22]:
tmp.all_tasks()[0]

TypeError: 'set' object is not subscriptable

In [7]:
loop.stop()

NameError: name 'loop' is not defined